# Import libraries
U know... just importing required libraries.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import csv

# Setup the csv path of a dataset
Please cheak the csv sample form from 'Dataset/csv/wiki_dataset.csv' to avoid unexpected results OuOb.

In [ ]:
# path = '../../Results/CLIP_Zeroshot_ALL.csv'
# path = '../../Results/YearCLIPv3.csv'
# path = '../../Results/NumCLIPv1.csv'

# path = '../../Results/4o_mini.csv'
# path = '../../Results/Gemini1.5.csv'
# path = '../../Results/Claude3.csv'
# path = '../../Results/Grok2.csv'

# path = '../../Results/VLMs/cogvlm2-llama3-chat-19B.csv'
# path = '../../Results/VLMs/gemma-3-4b-it.csv'
# path = '../../Results/VLMs/gemma-3-12b-it.csv'
# path = '../../Results/VLMs/glm-4v-9b.csv'
# path = '../../Results/VLMs/InternVL2-2B.csv'
# path = '../../Results/VLMs/InternVL2-4B.csv'
# path = '../../Results/VLMs/InternVL2-8B.csv'
# path = '../../Results/VLMs/InternVL2-26B.csv'
# path = '../../Results/VLMs/InternVL3-2B.csv'
# path = '../../Results/VLMs/InternVL3-8B.csv'
# path = '../../Results/VLMs/InternVL3-9B.csv'
# path = '../../Results/VLMs/llava-v16-vicuna-7b-hf.csv'
# path = '../../Results/VLMs/llava-v16-vicuna-13b-hf.csv'
# path = '../../Results/VLMs/llava15-7B-hf.csv'
# path = '../../Results/VLMs/llava15-13B-hf.csv'
# path = '../../Results/VLMs/MiniCPM-V-2_6.csv'
# path = '../../Results/VLMs/Qwen25VL-3B.csv'
# path = '../../Results/VLMs/Qwen25VL-7B.csv'
path = '../../Results/VLMs/Qwen25VL-32B.csv'
df = pd.read_csv(path)

# Show the comparison
Scatter the points (prediction , ground-truth) and draw a ideal red line.

In [ ]:
pred = df['pred'].to_numpy()
gt = df['year'].to_numpy()

plt.figure(figsize=(6,6))
plt.scatter(pred, gt, alpha=0.5)
plt.plot(pred, pred, color='red')

plt.xlim(800, 2200)
plt.ylim(800, 2200)

plt.title(f"Prediction Error of Year Built, MAE = {np.mean(np.abs(pred-gt)):.2f}")
plt.xlabel('Prediction (Year Built)')
plt.ylabel('Ground Truth (Year Built)')

plt.show()

# Show the difference
Plot a histogram to visualize the difference between prediction and ground-truth.

In [ ]:
errors = df['diff'].to_numpy()

plt.hist(errors, bins=50, color='blue', edgecolor='black')

plt.title(f"Histogram of Prediction Error of Year Built, MAE = {np.mean(np.abs(errors)):.2f}")
plt.xlabel('Prediction Error (Years)')
plt.ylabel('Count')

plt.show()

In [ ]:
errors = df['diff'].to_numpy()

plt.hist(errors, bins=200, color='blue')

plt.title(f"Histogram of Prediction Error of Year Built, MAE = {np.mean(np.abs(errors)):.2f}")
plt.xlabel('Prediction Error (Years)')
plt.ylabel('Count')

plt.xlim(-200, 200)

plt.show()

In [ ]:
errors = df['diff'].to_numpy()

plt.hist(errors, bins=range(-200,200+1))

plt.title(f"Histogram of Prediction Error of Year Built, MAE = {np.mean(np.abs(errors)):.2f}")
plt.xlabel('Prediction Error (Years)')
plt.ylabel('Count')

plt.show()

# Show Classification Accuracy
Predefined bins = [0, 1150, 1400, 1600, 1800, 1900, 1950, float('inf')]

In [ ]:
bins = [0, 1150, 1400, 1600, 1800, 1900, 1950, float('inf')]
labels = list(range(7))

df['year_class'] = pd.cut(df['year'], bins=bins, labels=labels, right=True, include_lowest=True)
df['pred_class'] = pd.cut(df['pred'], bins=bins, labels=labels, right=True, include_lowest=True)

correct_predictions = (df['year_class'] == df['pred_class']).sum()
total_predictions = len(df)
accuracy = correct_predictions / total_predictions
print(f'CLS Accuracy：{accuracy:.2%}')

# TOP100 worst prediction
Display top100 prediction that miss the most.

In [ ]:
def show_worst_pred(top_100_df):
    img_folder = '../../Dataset/images/'

    fig, axes = plt.subplots(10, 10, figsize=(20, 20))
    fig.suptitle("TOP 100 Year Prediction Errors\n", fontsize=20)

    for idx, (index, row) in enumerate(top_100_df.iterrows()):
        ax = axes[idx // 10, idx % 10]
        img = mpimg.imread(img_folder + row['name'])
        ax.imshow(img)
        ax.set_title(f"Year: {row['year']}, Pred: {row['pred']}", fontsize=8)
        ax.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
abs_df = df.copy()
abs_df['diff'] = abs_df['diff'].abs()

show_worst_pred(abs_df.nlargest(100, 'diff'))
# show_worst_pred(df.nsmallest(100, 'diff'))
# show_worst_pred(df.nlargest(100, 'diff'))

# Interval Performance
Compute the accuracy within different intervals.

In [ ]:
abs_errors = np.array(abs_df['diff'].to_numpy())
interval = [5,20,50,100]

for i in interval:
    print(f"[{i}] {np.sum(abs_errors <= i) / len(abs_errors) * 100:.2f}%")

# Popularity (Views) and Prediction
It indicates whether the predictions are susceptible to the building's popularity.

We assume that the more famous a building is, the higher the number of views on the website will be.

In [ ]:
test_df = pd.read_csv('../../Dataset/csv/test.csv')

plt.figure(figsize=(10, 6))
plt.scatter(test_df['Views'], abs_df['diff'], alpha=0.5)

plt.xscale('log')
plt.title('Relation between Views and Prediction Error')
plt.xlabel('Views (Times)')
plt.ylabel('Prediction Error (Years)')
plt.grid(True)
plt.show()

This shows the accuracy within 5 years in each interval of views.

In [ ]:
bins = [0, 1e2, 1e3, 1e4, 1e5, np.inf]
labels = ['<1e2', '1e2-1e3', '1e3-1e4', '1e4-1e5', '>1e5']

test_df['Interval'] = pd.cut(test_df['Views'], bins=bins, labels=labels, right=False)
result = abs_df[abs_df['diff'] <= 5].groupby(test_df['Interval'],observed=True).size()
result /= test_df.groupby('Interval',observed=True).size()
result = result.apply(lambda x: f'{x * 100:.2f}%')

print(result)


# All Results

In [ ]:
print(f"MAE = {np.mean(np.abs(pred-gt)):.2f}\n")
print(f"CLS Acc = {accuracy:.2%}\n")

for i in interval:
    print(f"[{i}] {np.sum(abs_errors <= i) / len(abs_errors) * 100:.2f}%")

print()
print(result)